# 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DATA_PATH = "/content/drive/MyDrive/data/"
SEED = 42

import torch
import numpy as np
import random
import os
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

df = pd.read_csv(f"{DATA_PATH}titanic_train.csv")

# 결측치 처리
df.age = df.age.fillna(df.age.median())
df.fare = df.fare.fillna(df.fare.median())
df.cabin = df.cabin.fillna("UNK")
df.embarked = df.embarked.fillna(df.embarked.mode()[0])

# 학습에 바로 사용가능한 특성
cols = ["pclass","age","sibsp","parch","fare"]
features = df[cols]

# 범주형 one-hot encoding
cols = ["gender","embarked"]
enc = OneHotEncoder()
tmp = pd.DataFrame(
    enc.fit_transform(df[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
features = pd.concat([features,tmp],axis=1) # 특성
y_train = df["survived"].to_numpy() # 정답값

# 스케일링
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(features)

x_train.shape , y_train.shape

Mounted at /content/drive


((916, 10), (916,))

# 재현성 함수(Reproduction)

In [ ]:
def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# 데이터셋 클래스 구현
- pytorch에서 미니 배치 학습단위로 학습을 수행하기 위해 다음과 같은 클래스를 활용해야 한다.

- Dataset
  - 학습 데이터와 정답데이터를 인덱싱을 통해 반환할 수 있는 클래스(먼저 구현해야됨) > train, target

- DataLoader
  - 우리가 지정한 배치 사이즈만큼 배치단위로 데이터를 반환할 수 있게 하는 클래스

- Dataset 클래스 만들기

In [ ]:
class TitanicDataset(torch.utils.data.Dataset): # 데이터셋 클래스
  def __init__(self, x, y = None): # 객체 생성시 입력과 정답 데이터를 인스턴스 변수안에 저장하는 역할
    self.x = x
    self.y = y
    if self.y is not None: # y값이 있을 경우에만 불러오기
      self.y = y.reshape(-1,1) # 데이터 불러오기에서 정답데이터 생성 안할 경우

  def __len__(self): # 샘플 수를 반환해주는 역할
    # return self.x.shape[0]
    return len(self.x)

  def __getitem__(self, idx): # 인덱싱을 통해 입력과 정답데이터를 반환할 수 있게 해주는 역할
    item = { }
    item['x'] = torch.Tensor(self.x[idx]) # 자료형으로 가져오기

    if self.y is not None:
      item['y'] = torch.Tensor(self.y[idx])
    return item

- Dataset test

In [ ]:
dt= TitanicDataset(x_train,y_train)
display(dt[0]) # dt에 값이 제대로 들어왔는지 확인하기 인덱싱의 범위 > -len(x_train) ~ len(x_train)-1
len(dt)

{'x': tensor([0.0000, 0.8873, 0.0000, 0.0000, 0.0966, 0.0000, 1.0000, 1.0000, 0.0000,
         0.0000]),
 'y': tensor([0.])}

916

- DataLoader
  - 첫 번째 인수 : 데이터셋 객체 전달(dt)
  - 배치 사이즈 지정
  - shuffle 여부 지정

In [ ]:
# torch.utils.data.DataLoader: 인덱싱+슬라이싱 사용 x
dl = torch.utils.data.DataLoader(dt, batch_size = 2, shuffle = False)
batch = next(iter(dl)) # iterable한 객체로 만들기
batch

{'x': tensor([[0.0000, 0.8873, 0.0000, 0.0000, 0.0966, 0.0000, 1.0000, 1.0000, 0.0000,
          0.0000],
         [1.0000, 0.4238, 0.0000, 0.0000, 0.0157, 0.0000, 1.0000, 0.0000, 0.0000,
          1.0000]]),
 'y': tensor([[0.],
         [0.]])}

In [ ]:
batch['x']

tensor([[0.0000, 0.8873, 0.0000, 0.0000, 0.0966, 0.0000, 1.0000, 1.0000, 0.0000,
         0.0000],
        [1.0000, 0.4238, 0.0000, 0.0000, 0.0157, 0.0000, 1.0000, 0.0000, 0.0000,
         1.0000]])

In [ ]:
len(batch)

2

In [ ]:
for batch in dl:
  print(batch)
  break

{'x': tensor([[0.0000, 0.8873, 0.0000, 0.0000, 0.0966, 0.0000, 1.0000, 1.0000, 0.0000,
         0.0000],
        [1.0000, 0.4238, 0.0000, 0.0000, 0.0157, 0.0000, 1.0000, 0.0000, 0.0000,
         1.0000]]), 'y': tensor([[0.],
        [0.]])}


## torch.nn.Linear 레이어
- 입력받은 데이터를 선형변환 해주는 레이어
  - 첫 번째 : 입력 피처 개수
  - 두 번째 : 출력 개수(노드 수)
- FC(Fully-Connected) layer라고 함 > 이전 출력과 완전 연결되었다는 의미

In [ ]:
x_train.shape[1]

10

In [ ]:
reset_seeds(SEED) # 시드고정
hidden_layer = torch.nn.Linear(x_train.shape[1],4) # 모델 객체, 입력데이터 피처 개수: 10, 출력 예측값 개수: 4
x = hidden_layer(batch['x']) # model()
x

tensor([[ 0.3429,  0.2720, -0.2034,  0.3389],
        [ 0.0509,  0.5410, -0.4339,  0.4020]], grad_fn=<AddmmBackward0>)

# 활성화 함수(Activation function)
- 입력된 데이터의 가중 합을 출력 신호로 변환하는 함수 > 각 학습데이터,테스트데이터
- 비선형성 추가

## 시그모이드 함수(Sigmoid) > BCE
- 0 ~ 1 사이의 값을 갖는 함수

In [ ]:
sig = torch.nn.Sigmoid()
sig(x)

tensor([[0.5849, 0.5676, 0.4493, 0.5839],
        [0.5127, 0.6320, 0.3932, 0.5992]], grad_fn=<SigmoidBackward0>)

## 탄젠트 함수(Hyperbolic Tangent)
- -1 ~ 1 사이의 값을 갖는 함수

In [ ]:
tanh = torch.nn.Tanh()
tanh(x)

tensor([[ 0.3301,  0.2655, -0.2006,  0.3265],
        [ 0.0509,  0.4937, -0.4086,  0.3817]], grad_fn=<TanhBackward0>)

## ReLU(Rectified Linear Unit) > BCE
- 입력이 0이상이면 입력을 그대로 출력하고, 음수이면 0을 출력하는 함수

In [ ]:
relu = torch.nn.ReLU()
relu(x)

tensor([[0.3429, 0.2720, 0.0000, 0.3389],
        [0.0509, 0.5410, 0.0000, 0.4020]], grad_fn=<ReluBackward0>)

## LeakyReLU > BCE
- ReLU의 변형된 함수
- Dying ReLU 현상을 해결하기 위해 나온 함수

In [ ]:
lk_relu = torch.nn.LeakyReLU()
lk_relu(x)

tensor([[ 0.3429,  0.2720, -0.0020,  0.3389],
        [ 0.0509,  0.5410, -0.0043,  0.4020]], grad_fn=<LeakyReluBackward0>)

## PReLU(Parametric ReLU)
- Leakly ReLU와 유사하지만 음수 입력에 대한 경사를 학습을 통해 업데이트

In [ ]:
prelu = torch.nn.PReLU()
prelu(x)

tensor([[ 0.3429,  0.2720, -0.0508,  0.3389],
        [ 0.0509,  0.5410, -0.1085,  0.4020]], grad_fn=<PreluKernelBackward0>)

## ELU(Exponential Linear Unit)
- 입력이 0 이하일 경우 지수 함수를 이용하여 부드럽게 깎아준다.
- ReLU의 모든 장점 포함
- exp를 계산하는 비용 발생

In [ ]:
elue = torch.nn.ELU()
elue(x)

tensor([[ 0.3429,  0.2720, -0.1840,  0.3389],
        [ 0.0509,  0.5410, -0.3520,  0.4020]], grad_fn=<EluBackward0>)

## 소프트맥스 함수(Softmax) > CE
- 입력받은 값들을 출력으로 0~1사이의 값들로 모두 정규화하며 출력 값들의 총합은  항상 1이 되는 특성을 가진 함수
- 다중 분류문제에서 확률값

In [ ]:
softmax = torch.nn.Softmax(dim = 1)
softmax(x)

tensor([[0.2852, 0.2657, 0.1651, 0.2840],
        [0.2142, 0.3496, 0.1319, 0.3043]], grad_fn=<SoftmaxBackward0>)

# 배치 정규화(Batch Normalization)
- 배치단위로 평균과 분산을 이용해 정규화
- 딥러닝 학습의 안정성과 속도를 크게 향상시키는 데 기여

- 주요 특징
  - 학습을 가속화하고 불안정한 경사를 완화하여 학습이 안정적
  - 더 큰 학습률을 사용할 수 있어 수렴 속도를 높이는 데 기여
  - 과적합을 방지하는 효과
  - 미니 배치 크기가 작은 경우, 통계의 변동 영향 > 성능 저하

- 배치 정규화는 일반적으로 활성화 함수 이전에 넣음
  - ex) FC(Linear) > BN > H(활성화)

In [ ]:
# 음수 계산값을 포함한 leakyReLU,PReLU, ELU와 조합이 우세
bn = torch.nn.BatchNorm1d(x.shape[1])
bn(x)

tensor([[ 0.9998, -0.9997,  0.9996, -0.9950],
        [-0.9998,  0.9997, -0.9996,  0.9950]],
       grad_fn=<NativeBatchNormBackward0>)

In [ ]:
x[0]
#  bn(x[:1]) ## 배치 개수가 1개 일때 에러

tensor([ 0.3429,  0.2720, -0.2034,  0.3389], grad_fn=<SelectBackward0>)

In [ ]:
x

tensor([[ 0.3429,  0.2720, -0.2034,  0.3389],
        [ 0.0509,  0.5410, -0.4339,  0.4020]], grad_fn=<AddmmBackward0>)

In [ ]:
display(x[:1])
x[:1].shape

tensor([[ 0.3429,  0.2720, -0.2034,  0.3389]], grad_fn=<SliceBackward0>)

torch.Size([1, 4])

# Dropout
- 신경망에서 은닉층의 각 노드에 대해서 0~1 사이 확률로 랜덤하게 0으로 대체

- 학습 중 특정 뉴런을 랜덤하게 비활성화하여 신경망이 특정 뉴런에 과도하게 의존하는 것을 방지, 네트워크의 일반화 성능을 높이는 데 기여

- 장점
  - 빠른 학습(계산 비용이 작아짐)
  - 과적합(overfitting)을 방지
  - 일반적으로 활성화 함수 다음 배치

- ex) FC > H > DO

In [ ]:
dropout = torch.nn.Dropout(0.5) # 0.5 확률로 각 노드값 > 0 대체

In [ ]:
x = lk_relu(x)
dropout(x) # 각 노드, 샘플에 대한 확률

tensor([[0.0000, 0.0000, -0.0000, 0.0000],
        [0.0000, 1.0819, -0.0000, 0.8041]], grad_fn=<MulBackward0>)

# torch.nn.Sequential
- 텐서가 여러 layer를 정해준 순서로 순차적으로 통과해야할 때 사용

In [ ]:
x_train.shape[1]

10

- 이진분류문제 > 출력 예측 개수: 1
- 다중분류문제 > 출력 예측 개수: 복수 개수

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(x_train.shape[1], 8), #FC, 선형변환, 입력데이터 피처 개수: 10, 출력 예측 개수: 8
    torch.nn.ReLU(),
    torch.nn.Linear(8, 4), # 입력데이터 피처 개수: 8, 출력 예측 개수: 4
    torch.nn.ReLU(),
    torch.nn.Linear(4, 1) # 입력데이터 피처 개수: 4, 출력 예측 개수: 1
)
model

Sequential(
  (0): Linear(in_features=10, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=4, bias=True)
  (3): ReLU()
  (4): Linear(in_features=4, out_features=1, bias=True)
)

In [ ]:
model(batch['x'])

tensor([[-0.1180],
        [-0.1115]], grad_fn=<AddmmBackward0>)

In [ ]:
# Module 내 forward 내장
class Net(torch.nn.Module):
  def __init__(self, in_features): # 신경망 레이어 클래스를 실행하여 객체를 생성
    super().__init__() # 부모클래스에 __init__메서드를 필수적으로 실행해줘야한다.
    self.seq = torch.nn.Sequential( # 순전파
      torch.nn.Linear(in_features, 8),
      torch.nn.ReLU(),
      torch.nn.Linear(8, 4),
      torch.nn.ReLU(),
      torch.nn.Linear(4, 1)
    )

  def forward(self, x): # 생성된 신경망 객체를 사용, 필수적으로 구현해야함
    return self.seq(x)


In [ ]:
model = Net(x_train.shape[1]) # in_features
model(batch['x']) # forward 메서드 실행

tensor([[-0.1826],
        [-0.2288]], grad_fn=<AddmmBackward0>)

# 모델구조 확인하기

- torchinfo
  - 첫 번째 인수 : model(신경망)
  - 두 번째 인수 : 배치사이즈 + 피처개수 튜플 전달

In [ ]:
%pip install torchinfo

In [ ]:
import torchinfo
torchinfo.summary(model, (2, x_train.shape[1]))

Layer (type:depth-idx)                   Output Shape              Param #
Net                                      [2, 1]                    --
├─Sequential: 1-1                        [2, 1]                    --
│    └─Linear: 2-1                       [2, 8]                    88
│    └─ReLU: 2-2                         [2, 8]                    --
│    └─Linear: 2-3                       [2, 4]                    36
│    └─ReLU: 2-4                         [2, 4]                    --
│    └─Linear: 2-5                       [2, 1]                    5
Total params: 129
Trainable params: 129
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

# 모델 저장 및 불러오기

In [ ]:
model.state_dict()

OrderedDict([('seq.0.weight',
              tensor([[-0.1514, -0.2574,  0.2652, -0.1266,  0.0838, -0.1097,  0.0257,  0.2948,
                        0.1457, -0.2740],
                      [ 0.1255,  0.3002,  0.0832,  0.2120,  0.3118, -0.0485,  0.0656, -0.2198,
                       -0.0652,  0.2342],
                      [ 0.1621, -0.2001, -0.2536, -0.2161, -0.3121, -0.2440, -0.0782,  0.2134,
                        0.0529, -0.2405],
                      [-0.2537,  0.1573, -0.2352, -0.0389,  0.1517, -0.1464, -0.0345, -0.0275,
                       -0.0748, -0.1603],
                      [-0.2819, -0.2556, -0.1693,  0.3054, -0.1527, -0.2124,  0.0767,  0.0872,
                        0.1733,  0.2404],
                      [ 0.1761, -0.3135,  0.0280,  0.1916, -0.0292, -0.1863,  0.3015, -0.1183,
                       -0.1800, -0.2851],
                      [ 0.0141,  0.1401,  0.0700,  0.0625, -0.2398, -0.2953,  0.0056,  0.2883,
                        0.1824, -0.1841],
           

- 학습 완료된 모델 가중치 저장하기
  - save 함수
  - 첫 번째 인수: .state_dict()
  - 두 번째 인수: 저장명

In [ ]:
torch.save(model.state_dict(), 'model.pt')

- 저장된 가중치 불러오기
  - load 함수
  - weights_only = True

In [ ]:
state_dict = torch.load('model.pt', weights_only = True)
state_dict

OrderedDict([('seq.0.weight',
              tensor([[-0.1514, -0.2574,  0.2652, -0.1266,  0.0838, -0.1097,  0.0257,  0.2948,
                        0.1457, -0.2740],
                      [ 0.1255,  0.3002,  0.0832,  0.2120,  0.3118, -0.0485,  0.0656, -0.2198,
                       -0.0652,  0.2342],
                      [ 0.1621, -0.2001, -0.2536, -0.2161, -0.3121, -0.2440, -0.0782,  0.2134,
                        0.0529, -0.2405],
                      [-0.2537,  0.1573, -0.2352, -0.0389,  0.1517, -0.1464, -0.0345, -0.0275,
                       -0.0748, -0.1603],
                      [-0.2819, -0.2556, -0.1693,  0.3054, -0.1527, -0.2124,  0.0767,  0.0872,
                        0.1733,  0.2404],
                      [ 0.1761, -0.3135,  0.0280,  0.1916, -0.0292, -0.1863,  0.3015, -0.1183,
                       -0.1800, -0.2851],
                      [ 0.0141,  0.1401,  0.0700,  0.0625, -0.2398, -0.2953,  0.0056,  0.2883,
                        0.1824, -0.1841],
           